### Importing required libraries

In [2]:
import nltk
import json
import pickle
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from nltk.stem import WordNetLemmatizer
import random

In [3]:
#loading the json intent file
data_file = open('intents.json').read()
intents = json.loads(data_file)

### Data preprocessing

In [4]:
#initializing lemmatizer
lemmatizer = WordNetLemmatizer()

In [5]:
#assigning empty list variables to append class labels,all words as a corpus,tokenized sentences and its tags
words=[]
classes = []
documents = []

In [ ]:
#characters to be ignored while lemmatizing
ignore_words = ['?', '!']

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenizing each word
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        
        #adding tokenized words,tag of the words in a tuple
        documents.append((tokens, intent['tag']))
        
        # adding the distinct classes to know how many class labels present 
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# lemmatizing, lowering ,sorting each word and removing duplicates using set
words = [lemmatizer.lemmatize(w.lower())for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), "documents")
print()

# classes = intents
print(len(classes), "classes", classes)
print()

# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

43 documents

10 classes ['caring', 'goodbye', 'greeting', 'help', 'hobbies', 'investigating', 'joke', 'personel_info', 'smalltalk', 'thanks']

77 unique lemmatized words ["'ll", "'m", "'s", 'a', 'about', 'afternoon', 'any', 'are', 'back', 'be', 'can', 'care', 'catch', 'confused', 'do', 'doing', 'enjoy', 'evening', 'explain', 'family', 'favorite', 'fine', 'for', 'free', 'from', 'fun', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'hey', 'hi', 'hobby', 'how', 'i', 'in', 'interest', 'joke', 'later', 'like', 'look', 'lost', 'me', 'morning', 'movie', 'music', "n't", 'need', 'of', 'old', 'outside', 'say', 'see', 'so', 'stuck', 'take', 'talk', 'tell', 'thank', 'thanks', 'that', 'there', 'this', 'time', 'to', 'travel', 'understand', 'wa', 'want', 'what', 'where', 'who', 'work', 'you', 'your']


In [ ]:
#saving the word corpus,class labels of our collection as a pickle file for later purpose
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [42]:
# creating our training data
training = []

# creating an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initializing our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatizing each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
# create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
# shuffling our features and converting into array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data created


C:\Users\navin\AppData\Local\Temp\ipykernel_16256\2769392415.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


### model building

In [43]:
# Creating sequential model with 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
#1st layer
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))

#adding dropout layer1 to avoid over fitting
model.add(Dropout(0.5))
#2nd layer
model.add(Dense(64, activation='relu')

#adding dropout layer2
model.add(Dropout(0.5))

#3rd layer
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compiling model with adam optimizer which produces great results
model.compile(loss='categorical_crossentropy', optimizer= tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=199, batch_size=8, verbose=1)

Epoch 1/199
6/6 [==============================] - 0s 1ms/step - loss: 2.3466 - accuracy: 0.0233
Epoch 2/199
6/6 [==============================] - 0s 800us/step - loss: 2.3008 - accuracy: 0.1628
Epoch 3/199
6/6 [==============================] - 0s 600us/step - loss: 2.2907 - accuracy: 0.0698
Epoch 4/199
6/6 [==============================] - 0s 800us/step - loss: 2.2650 - accuracy: 0.1163
Epoch 5/199
6/6 [==============================] - 0s 800us/step - loss: 2.2014 - accuracy: 0.2093
Epoch 6/199
6/6 [==============================] - 0s 600us/step - loss: 2.1536 - accuracy: 0.2558
Epoch 7/199
6/6 [==============================] - 0s 800us/step - loss: 2.1766 - accuracy: 0.2791
Epoch 8/199
6/6 [==============================] - 0s 800us/step - loss: 2.1540 - accuracy: 0.2326
Epoch 9/199
6/6 [==============================] - 0s 800us/step - loss: 2.1547 - accuracy: 0.1860
Epoch 10/199
6/6 [==============================] - 0s 800us/step - loss: 2.0805 - accuracy: 0.3953
Epoch 11/19

6/6 [==============================] - 0s 600us/step - loss: 0.0645 - accuracy: 1.0000
Epoch 165/199
6/6 [==============================] - 0s 600us/step - loss: 0.0339 - accuracy: 1.0000
Epoch 166/199
6/6 [==============================] - 0s 800us/step - loss: 0.0517 - accuracy: 1.0000
Epoch 167/199
6/6 [==============================] - 0s 600us/step - loss: 0.0393 - accuracy: 1.0000
Epoch 168/199
6/6 [==============================] - 0s 800us/step - loss: 0.1102 - accuracy: 0.9767
Epoch 169/199
6/6 [==============================] - 0s 800us/step - loss: 0.0767 - accuracy: 0.9767
Epoch 170/199
6/6 [==============================] - 0s 600us/step - loss: 0.1221 - accuracy: 0.9535
Epoch 171/199
6/6 [==============================] - 0s 800us/step - loss: 0.0314 - accuracy: 1.0000
Epoch 172/199
6/6 [==============================] - 0s 800us/step - loss: 0.0596 - accuracy: 1.0000
Epoch 173/199
6/6 [==============================] - 0s 600us/step - loss: 0.0484 - accuracy: 1.0000
Epoc

### saving the trained model.

In [44]:
model.save('chatbot_model.h5')